In [1]:
import os
import json
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding

In [2]:
data_directory = "../Data2/"
data_file = "Data_Tunes.txt"  ## contains 744 Tunes Sample
charIndex_json = "char_to_index.json"
model_weights_directory = '../Data2/Model_Weights/'
BATCH_SIZE = 32
SEQ_LENGTH = 64

In [3]:
def read_batches(all_chars, unique_chars):
    length = all_chars.shape[0]
    batch_chars = int(length / BATCH_SIZE) #155222/16 = 9701
    
    for start in range(0, batch_chars - SEQ_LENGTH, 64):  #(0, 9637, 64)  #it denotes number of batches. It runs everytime when
        #new batch is created. We have a total of 151 batches.
        X = np.zeros((BATCH_SIZE, SEQ_LENGTH))    #(32, 64)
        Y = np.zeros((BATCH_SIZE, SEQ_LENGTH, unique_chars))   #(32, 64, 87)
        for batch_index in range(0, 32):  #it denotes each row in a batch.  
            for i in range(0, 64):  #it denotes each column in a batch. Each column represents each character means 
                #each time-step character in a sequence.
                X[batch_index, i] = all_chars[batch_index * batch_chars + start + i]
                Y[batch_index, i, all_chars[batch_index * batch_chars + start + i + 1]] = 1 #here we have added '1' because the
                #correct label will be the next character in the sequence. So, the next character will be denoted by
                #all_chars[batch_index * batch_chars + start + i] + 1. 
        yield X, Y

In [4]:
def built_model(batch_size, seq_length, unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (batch_size, seq_length), name = "embd_1")) 
    
    model.add(LSTM(256, return_sequences = True, stateful = True, name = "lstm_first"))
    model.add(Dropout(0.2, name = "drp_1"))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.3))
    
    model.add(TimeDistributed(Dense(unique_chars)))
    model.add(Activation("softmax"))
    
    model.load_weights("../Data/Model_Weights/Weights_80.h5", by_name = True)
    
    return model

In [5]:
def training_model(data, epochs = 200):
    #mapping character to index
    char_to_index = {ch: i for (i, ch) in enumerate(sorted(list(set(data))))}
    print("Number of unique characters in our whole tunes database = {}".format(len(char_to_index))) #87
    
    with open(os.path.join(data_directory, charIndex_json), mode = "w") as f:
        json.dump(char_to_index, f)
        
    index_to_char = {i: ch for (ch, i) in char_to_index.items()}
    unique_chars = len(char_to_index)
    
    model = built_model(BATCH_SIZE, SEQ_LENGTH, unique_chars)
    model.summary()
    model.compile(loss = "categorical_crossentropy", optimizer = "Adamax", metrics = ["accuracy"])
    
    all_characters = np.asarray([char_to_index[c] for c in data], dtype = np.int32)
    print("Total number of characters = "+str(all_characters.shape[0])) #155222
    
    epoch_number, loss, accuracy = [], [], []
    
    for epoch in range(epochs):
        print("Epoch {}/{}".format(epoch+1, epochs))
        final_epoch_loss, final_epoch_accuracy = 0, 0
        epoch_number.append(epoch+1)
        
        for i, (x, y) in enumerate(read_batches(all_characters, unique_chars)):
            final_epoch_loss, final_epoch_accuracy = model.train_on_batch(x, y) #check documentation of train_on_batch here: https://keras.io/models/sequential/
            print("Batch: {}, Loss: {}, Accuracy: {}".format(i+1, final_epoch_loss, final_epoch_accuracy))
            #here, above we are reading the batches one-by-one and train our model on each batch one-by-one.
        loss.append(final_epoch_loss)
        accuracy.append(final_epoch_accuracy)
        
        #saving weights after every 10 epochs
        if (epoch + 1) % 10 == 0:
            if not os.path.exists(model_weights_directory):
                os.makedirs(model_weights_directory)
            model.save_weights(os.path.join(model_weights_directory, "Weights_{}.h5".format(epoch+1)))
            print('Saved Weights at epoch {} to file Weights_{}.h5'.format(epoch+1, epoch+1))
    
    #creating dataframe and record all the losses and accuracies at each epoch
    log_frame = pd.DataFrame(columns = ["Epoch", "Loss", "Accuracy"])
    log_frame["Epoch"] = epoch_number
    log_frame["Loss"] = loss
    log_frame["Accuracy"] = accuracy
    log_frame.to_csv("../Data2/log.csv", index = False)

In [ ]:
file = open(os.path.join(data_directory, data_file), mode = 'r')
data = file.read()
file.close()
if __name__ == "__main__":
    training_model(data)

Number of unique characters in our whole tunes database = 92
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embd_1 (Embedding)           (32, 64, 512)             47104     
_________________________________________________________________
lstm_first (LSTM)            (32, 64, 256)             787456    
_________________________________________________________________
drp_1 (Dropout)              (32, 64, 256)             0         
_________________________________________________________________
lstm_1 (LSTM)                (32, 64, 256)             525312    
_________________________________________________________________
dropout_1 (Dropout)          (32, 64, 256)             0         
_________________________________________________________________
lstm_2 (LSTM)                (32, 64, 256)             525312    
___________________________________________________________

C:\Users\SACHIN\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Batch: 1, Loss: 4.521768569946289, Accuracy: 0.0146484375
Batch: 2, Loss: 4.500132083892822, Accuracy: 0.125
Batch: 3, Loss: 4.397945404052734, Accuracy: 0.123046875
Batch: 4, Loss: 3.98176908493042, Accuracy: 0.130859375
Batch: 5, Loss: 3.7812066078186035, Accuracy: 0.12548828125
Batch: 6, Loss: 3.590672016143799, Accuracy: 0.1259765625
Batch: 7, Loss: 3.573263168334961, Accuracy: 0.10791015625
Batch: 8, Loss: 3.520186424255371, Accuracy: 0.08837890625
Batch: 9, Loss: 3.7517571449279785, Accuracy: 0.0791015625
Batch: 10, Loss: 3.6481857299804688, Accuracy: 0.07958984375
Batch: 11, Loss: 3.5272161960601807, Accuracy: 0.09521484375
Batch: 12, Loss: 3.4136385917663574, Accuracy: 0.1171875
Batch: 13, Loss: 3.3235745429992676, Accuracy: 0.13037109375
Batch: 14, Loss: 3.515815019607544, Accuracy: 0.10888671875
Batch: 15, Loss: 3.4710474014282227, Accuracy: 0.10009765625
Batch: 16, Loss: 3.6054911613464355, Accuracy: 0.09716796875
Batch: 17, Loss: 3.572608232498169, Accuracy: 0.095703125
Bat

Batch: 139, Loss: 3.022627830505371, Accuracy: 0.18212890625
Batch: 140, Loss: 2.9675331115722656, Accuracy: 0.20166015625
Batch: 141, Loss: 2.926487684249878, Accuracy: 0.20751953125
Batch: 142, Loss: 2.98268985748291, Accuracy: 0.19873046875
Batch: 143, Loss: 3.01859712600708, Accuracy: 0.1806640625
Batch: 144, Loss: 3.0294322967529297, Accuracy: 0.1884765625
Batch: 145, Loss: 2.9960551261901855, Accuracy: 0.19580078125
Batch: 146, Loss: 2.888007402420044, Accuracy: 0.212890625
Batch: 147, Loss: 2.943621873855591, Accuracy: 0.212890625
Batch: 148, Loss: 2.9333136081695557, Accuracy: 0.22802734375
Batch: 149, Loss: 2.9116742610931396, Accuracy: 0.212890625
Batch: 150, Loss: 2.9598922729492188, Accuracy: 0.21826171875
Batch: 151, Loss: 2.859642744064331, Accuracy: 0.22216796875
Batch: 152, Loss: 2.8918113708496094, Accuracy: 0.2255859375
Batch: 153, Loss: 2.873473882675171, Accuracy: 0.220703125
Batch: 154, Loss: 2.871687412261963, Accuracy: 0.21875
Batch: 155, Loss: 2.870089530944824,

Batch: 96, Loss: 2.337177276611328, Accuracy: 0.3818359375
Batch: 97, Loss: 2.282219409942627, Accuracy: 0.40869140625
Batch: 98, Loss: 2.3455958366394043, Accuracy: 0.3984375
Batch: 99, Loss: 2.2546539306640625, Accuracy: 0.38818359375
Batch: 100, Loss: 2.324881076812744, Accuracy: 0.384765625
Batch: 101, Loss: 2.296651840209961, Accuracy: 0.39404296875
Batch: 102, Loss: 2.2353978157043457, Accuracy: 0.4013671875
Batch: 103, Loss: 2.2952094078063965, Accuracy: 0.3876953125
Batch: 104, Loss: 2.3384246826171875, Accuracy: 0.388671875
Batch: 105, Loss: 2.2880301475524902, Accuracy: 0.39453125
Batch: 106, Loss: 2.2699694633483887, Accuracy: 0.39599609375
Batch: 107, Loss: 2.2874441146850586, Accuracy: 0.3935546875
Batch: 108, Loss: 2.249988079071045, Accuracy: 0.41650390625
Batch: 109, Loss: 2.2782764434814453, Accuracy: 0.3916015625
Batch: 110, Loss: 2.234407901763916, Accuracy: 0.40185546875
Batch: 111, Loss: 2.227210521697998, Accuracy: 0.40625
Batch: 112, Loss: 2.2532477378845215, Acc

...

Batch: 62, Loss: 0.49889832735061646, Accuracy: 0.84130859375
Batch: 63, Loss: 0.4747300148010254, Accuracy: 0.83935546875
Batch: 64, Loss: 0.516369640827179, Accuracy: 0.8369140625
Batch: 65, Loss: 0.4895305037498474, Accuracy: 0.8486328125
Batch: 66, Loss: 0.4865865409374237, Accuracy: 0.84814453125
Batch: 67, Loss: 0.49876686930656433, Accuracy: 0.837890625
Batch: 68, Loss: 0.4569424092769623, Accuracy: 0.8515625
Batch: 69, Loss: 0.5000662803649902, Accuracy: 0.8359375
Batch: 70, Loss: 0.4499408006668091, Accuracy: 0.85107421875
Batch: 71, Loss: 0.47717055678367615, Accuracy: 0.8359375
Batch: 72, Loss: 0.5356087684631348, Accuracy: 0.81787109375
Batch: 73, Loss: 0.4772317409515381, Accuracy: 0.85009765625
Batch: 74, Loss: 0.49537771940231323, Accuracy: 0.84375
Batch: 75, Loss: 0.4815453290939331, Accuracy: 0.8369140625
Batch: 76, Loss: 0.45240336656570435, Accuracy: 0.8505859375
Batch: 77, Loss: 0.4525792598724365, Accuracy: 0.86279296875
Batch: 78, Loss: 0.49264276027679443, Accura

Batch: 17, Loss: 0.46270471811294556, Accuracy: 0.8505859375
Batch: 18, Loss: 0.4861786663532257, Accuracy: 0.84375
Batch: 19, Loss: 0.5090154409408569, Accuracy: 0.83935546875
Batch: 20, Loss: 0.44144558906555176, Accuracy: 0.85693359375
Batch: 21, Loss: 0.49414026737213135, Accuracy: 0.84326171875
Batch: 22, Loss: 0.4467250108718872, Accuracy: 0.85107421875
Batch: 23, Loss: 0.4323917627334595, Accuracy: 0.85400390625
Batch: 24, Loss: 0.4777716398239136, Accuracy: 0.84130859375
Batch: 25, Loss: 0.4774099886417389, Accuracy: 0.84326171875
Batch: 26, Loss: 0.46107539534568787, Accuracy: 0.841796875
Batch: 27, Loss: 0.519924521446228, Accuracy: 0.8388671875
Batch: 28, Loss: 0.4679490625858307, Accuracy: 0.85107421875
Batch: 29, Loss: 0.5043957233428955, Accuracy: 0.83544921875
Batch: 30, Loss: 0.5014980435371399, Accuracy: 0.84130859375
Batch: 31, Loss: 0.5265881419181824, Accuracy: 0.83056640625
Batch: 32, Loss: 0.5308750867843628, Accuracy: 0.82275390625
Batch: 33, Loss: 0.481787890195

Batch: 153, Loss: 0.4852222204208374, Accuracy: 0.84423828125
Batch: 154, Loss: 0.47185438871383667, Accuracy: 0.8427734375
Batch: 155, Loss: 0.5355594158172607, Accuracy: 0.8271484375
Batch: 156, Loss: 0.4467182755470276, Accuracy: 0.8544921875
Batch: 157, Loss: 0.44302690029144287, Accuracy: 0.83984375
Batch: 158, Loss: 0.45552295446395874, Accuracy: 0.85693359375
Batch: 159, Loss: 0.4832078218460083, Accuracy: 0.83544921875
Batch: 160, Loss: 0.46466612815856934, Accuracy: 0.8486328125
Batch: 161, Loss: 0.504163384437561, Accuracy: 0.83642578125
Batch: 162, Loss: 0.4903491139411926, Accuracy: 0.837890625
Batch: 163, Loss: 0.480683296918869, Accuracy: 0.84375
Batch: 164, Loss: 0.5273023843765259, Accuracy: 0.83056640625
Batch: 165, Loss: 0.4844821095466614, Accuracy: 0.83740234375
Batch: 166, Loss: 0.5061507225036621, Accuracy: 0.83837890625
Batch: 167, Loss: 0.49291762709617615, Accuracy: 0.8447265625
Batch: 168, Loss: 0.45250996947288513, Accuracy: 0.8447265625
Batch: 169, Loss: 0.4

Batch: 109, Loss: 0.4742678999900818, Accuracy: 0.84130859375
Batch: 110, Loss: 0.4773295521736145, Accuracy: 0.8466796875
Batch: 111, Loss: 0.44087833166122437, Accuracy: 0.8515625
Batch: 112, Loss: 0.46137502789497375, Accuracy: 0.8515625
Batch: 113, Loss: 0.5065615177154541, Accuracy: 0.8251953125
Batch: 114, Loss: 0.5075063705444336, Accuracy: 0.83935546875
Batch: 115, Loss: 0.4759484827518463, Accuracy: 0.84326171875
Batch: 116, Loss: 0.4425850510597229, Accuracy: 0.861328125
Batch: 117, Loss: 0.44582200050354004, Accuracy: 0.84619140625
Batch: 118, Loss: 0.49049532413482666, Accuracy: 0.8369140625
Batch: 119, Loss: 0.48799529671669006, Accuracy: 0.837890625
Batch: 120, Loss: 0.4680337607860565, Accuracy: 0.84716796875
Batch: 121, Loss: 0.46164020895957947, Accuracy: 0.84765625
Batch: 122, Loss: 0.4579319953918457, Accuracy: 0.841796875
Batch: 123, Loss: 0.45053985714912415, Accuracy: 0.85693359375
Batch: 124, Loss: 0.4325264096260071, Accuracy: 0.8564453125
Batch: 125, Loss: 0.47

Batch: 64, Loss: 0.4957650303840637, Accuracy: 0.82958984375
Batch: 65, Loss: 0.5179043412208557, Accuracy: 0.822265625
Batch: 66, Loss: 0.48863834142684937, Accuracy: 0.84375
Batch: 67, Loss: 0.5140656232833862, Accuracy: 0.8359375
Batch: 68, Loss: 0.4264504313468933, Accuracy: 0.86083984375
Batch: 69, Loss: 0.4708821177482605, Accuracy: 0.84619140625
Batch: 70, Loss: 0.4409862160682678, Accuracy: 0.8505859375
Batch: 71, Loss: 0.4662485718727112, Accuracy: 0.84814453125
Batch: 72, Loss: 0.48853233456611633, Accuracy: 0.83349609375
Batch: 73, Loss: 0.48572391271591187, Accuracy: 0.84375
Batch: 74, Loss: 0.4791041314601898, Accuracy: 0.8466796875
Batch: 75, Loss: 0.4498368203639984, Accuracy: 0.8466796875
Batch: 76, Loss: 0.4274808466434479, Accuracy: 0.87255859375
Batch: 77, Loss: 0.455777645111084, Accuracy: 0.8583984375
Batch: 78, Loss: 0.49066802859306335, Accuracy: 0.8447265625
Batch: 79, Loss: 0.4709770679473877, Accuracy: 0.8515625
Batch: 80, Loss: 0.47901231050491333, Accuracy: 

In [23]:
log = pd.read_csv(os.path.join(data_directory, "log.csv"))
log

,Epoch,Loss,Accuracy
0,1,3.510531,0.140137
1,2,3.274780,0.162109
2,3,2.968642,0.208496
3,4,2.448124,0.333984
4,5,2.157264,0.395508
...,...,...,...
155,156,0.343946,0.883301
156,157,0.337188,0.896484
157,158,0.338072,0.895508
158,159,0.321451,0.891602
